# Problème 7 : Probabilités et poker

L'objectif de ce problème est d'estimer des probabilités pour le **poker**. En particulier, nous allons estimer les probabilités de gain des **mains** par la **méthode de Monte-Carlo**.

Le poker utilise un jeu de 52 cartes. Il y a 4 **couleurs** qui sont pique, coeur, carreau et trèfle. Attention, les couleurs ne sont pas rouge et noir, mais bien les quatre symboles. Pour chaque couleur, il y a 13 **hauteurs** de carte : 2, 3, 4, ..., 10, Valet, Dame, Roi et As .

## 1. Poker fermé

Au poker fermé chaque joueur reçoit 5 cartes d'un paquet de carte. 

Pour les questions mathématiques, on considère qu'une main est un *ensemble* de 5 cartes : l'ordre des cartes n'est pas pris en compte pour les dénombrements.

On rappelle que les coefficient binomial $\binom{n}{k}$ compte le nombre de façon de choisir $k$ éléments dans un ensemble avec $n$ éléments, et qu'on la formule, pour $0\leq k\leq n$ : $$\binom{n}{k} = \frac{n!}{k!(n-k)!}$$

Ecrire une fonction `binom(n,k)` qui calcule le coefficient binomial $\binom{n}{k}$:

In [1]:
def binom(n : int, k : int):
    if(n < k) :
        raise ValueError(" n doit être supérieur à k")
    elif(k < 0) :
        raise ValueError(" k doit être supérieur à 0")
        
    res =  1
    for i in range(k):
        res = res * (n - i)
        res = res // (i + 1)
        
    return res

Pour tous les calculs de probabilité, on donnera (comme pour toute les réponses de cette partie) la réponse sous forme d'une fraction irréductible, en justifiant. Vous pouvez utiliser votre fonction `binom` et votre sujet sur les fraction rationnelles si vous voulez pour aider.

In [2]:
from math import gcd

class Rat :
    
    def __init__(self, num : int, denom : int = 1) :
        
        if denom == 0 :
            raise ValueError("Dénominateur non-nul")
            
        pgcd = gcd(num, denom)
        num = num // pgcd
        denom = denom // pgcd
            
        if denom < 0 :
            denom *= -1
            num *= -1
        
        self.num = num
        self.denom = denom
        
    def __repr__(self) :
        return "Rat" + str((self.num, self.denom))

    def __str__(self) :
        if self.denom == 1 :
            return str(self.num)
        return str(self.num) + "/" + str(self.denom)

**Question :** Combien y a-t-il de mains de 5 cartes différentes ?

In [3]:
binom(52, 5)  #2 598 960

2598960

On considère que le paquet de carte est initalement *uniformément mélangé*, et donc que chaque main possible a autant de chance d'être celle obtenue par un joueur. 

**Question :** Quelle est la probabilité que la main obtenue soit {2-coeur, 7-coeur, 8-pique, Roi-carreau, As-pique} ? 

In [4]:
print(Rat(1, binom(52, 5)))

1/2598960


**Question :** Quelle est la probabilité que la main obtenue contienne 4 As (carré d'as) ?

In [5]:
print(Rat(48, binom(52, 5)))

1/54145


**Question :** Quelle est la probabilité que la main obtenue contienne un carré, soit 4 cartes de même hauteur ?

In [6]:
print(Rat(13 * 48, binom(52, 5)))

1/4165


Une **couleur** est une main de 5 cartes  de la même couleur (que des piques par exemple).

**Question :** Quelle est la probabilité que la main obtenue soit une couleur ?

In [7]:
print(Rat(4 * binom(13, 5), binom(52, 5)))
print(4 * binom(13, 5) / binom(52, 5))

33/16660
0.0019807923169267707


## 2. Les classes Carte et JeuDeCarte

* Ecrire une classe `Carte` pour représenter une carte à jouer. Une carte possède une couleur avec l'encodage : TREFLE = 0, CARREAU = 1, COEUR = 2, PIQUE = 3
et une hauteur qui est entre 2 et 14 avec : VALET = 11, DAME = 12, ROI = 13, AS = 14

- Le constructeur passera en argument la hauteur et la couleur : `__init__(self, hauteur, couleur)`
- Les méthodes `__str__(self)` et `__repr__(self)` retourneront la même chose, la chaîne de caractère pour décrire les cartes respectant le doctest fourni.
- La méthode `__lt__(self,other)` qui retourne vrai si la hauteur de `self` est strictement inférieure à la hauteur de `other` et faux sinon
- La méthode `__eq__(self, other)` qui teste si `self` est égale à `other` (même hauteur et même couleur)


In [23]:
TREFLE, CARREAU, COEUR, PIQUE = 0, 1, 2, 3 # on utilise des variables globales
VALET, DAME, ROI, AS = 11, 12, 13, 14

class Carte:
    """
    >>> Carte(10,0)
    10-Trefle
    >>> Carte(14,1)
    As-Carreau
    """
    def __init__(self, hauteur, couleur):
        self.hauteur = hauteur
        self.couleur = couleur
        
    def __lt__(self, other):
        if self.hauteur < other.hauteur :
            return True
        return False
    
    def __str__(self):
        hauteur = self.hauteur
        
        if self.hauteur == 11 :
            hauteur = 'Valet'
        elif self.hauteur == 12:
            hauteur = 'Dame'
        elif self.hauteur == 13 :
            hauteur = 'Roi'
        elif self.hauteur == 14 :
            hauteur = 'As'
            
        couleur = self.couleur    
        
        if self.couleur == 0 :
            couleur = 'Trefle'
        elif couleur == 1 :
            couleur = 'Carreau'
        elif self.couleur == 2 :
            couleur = 'Coeur'
        elif self.couleur == 3 :
            couleur = 'Pique'
            
        return str(hauteur) + "-" + str(couleur)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return (self.hauteur == other.hauteur) and (self.couleur == other.couleur)

* Vérifier avec doctest que cela fonctionne correctement.

In [63]:
import doctest
doctest.testmod()

**********************************************************************
File "__main__", line 5, in __main__.texas_holdem
Failed example:
    sorted(texas_holdem([Carte(2,0), Carte(7,0)], [Carte(3, 1), Carte(3,0), Carte(8,0), Carte(10, 2), Carte(11, 0)]))
Exception raised:
    Traceback (most recent call last):
      File "c:\users\selym\appdata\local\programs\python\python39-32\lib\doctest.py", line 1336, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest __main__.texas_holdem[0]>", line 1, in <module>
        sorted(texas_holdem([Carte(2,0), Carte(7,0)], [Carte(3, 1), Carte(3,0), Carte(8,0), Carte(10, 2), Carte(11, 0)]))
    TypeError: 'NoneType' object is not iterable
**********************************************************************
File "__main__", line 7, in __main__.texas_holdem
Failed example:
    sorted(texas_holdem([Carte(2,0), Carte(11,1)], [Carte(3, 1), Carte(3,0), Carte(8,0), Carte(10, 2), Carte(11, 0)]))
Exception raised:
    Traceba

TestResults(failed=2, attempted=18)

* Ecrire une classe `JeuDeCartes` avec :

- un constructeur sans argument `__init__(self)` qui créée un nouveau jeu de carte, contenant les 52 cartes, mélangées. On pourra utiliser `shuffle` de la bibliothèque `random` qui mélange uniformément un tableau come dans l'exemple ci-dessous
- une méthode `pioche(self)` qui pioche une carte : elle retourne la carte suivante du jeu de carte et l'enlève du jeu de carte
- une méthode `pioche_n(self)` qui pioche $n$ cartes : elle retourne la liste des cartes piochées et les enlève du jeu de carte

On ne demande pas de gérer les cas où il n'y a plus assez de cartes dans le jeu pour effectuer la (les) pioche(s)

In [10]:
from random import shuffle
T = list(range(20))
shuffle(T) # melange T, en modifiant T
print(T)

[9, 17, 18, 4, 5, 10, 14, 8, 15, 1, 0, 11, 2, 7, 16, 19, 12, 3, 6, 13]


In [11]:
from random import shuffle

class JeuDeCartes:
    def __init__(self):
        self.deck = []
        for hauteur in range(1, 15):
            for couleur in range(0, 4):
                self.deck.append(Carte(hauteur, couleur))
        shuffle(self.deck)
                
    def pioche(self):
        card = self.deck[0]
        self.deck.pop(0)
        return card

    def pioche_n(self, n):
        cards = []
        for _ in range(n):
            cards.append(self.deck[0])
            self.deck.pop(0)
        return cards

        
JC = JeuDeCartes()
M = JC.pioche_n(5)
print(M)

[8-Carreau, 8-Coeur, 6-Pique, 5-Pique, 7-Trefle]


## 3. Combinaisons de cartes

Au poker, on cherche à faire des combinaisons particulières de cartes. Voilà les combinaisons possibles de la moins forte à la plus forte :
- **Hauteur :** aucune des combinaisons ci-dessous (c'est la plus faible)
- **Paire :** deux cartes de même hauteur, et rien de mieux ci-dessous
- **Double paire :** deux groupes de deux cartes de même hauteur
- **Brelan :** trois cartes de la même hauteur
- **Quinte :** cinq cartes de hauteurs consécutives, par exemple [4-Coeur, 5-Trefle, 6-Trefle, 7-pique, 8-Coeur]. <font color=red>Attention :</font> l'as peut servir de 1 pour commencer une quinte [As-Coeur, 2-Trefle, 3-Trefle, 4-pique, 5-Coeur], ou d'As pour terminer une quinte [10-Pique, Valet-Pique, Dame-Pique, Roi-Carreau, As-Coeur]
- **Couleur :** toutes les cartes de la même couleur (par exemple, que des trèfles)
- **Full House:** un brelan et une paire
- **Carré :** quatre cartes de même hauteur
- **Quinte Flush :** quinte et couleur en même temps

Voir [l'article Wikipédia](https://fr.wikipedia.org/wiki/Main_au_poker) pour des illustrations.

* Ecrire une fonction `couleur(main)` qui prend en paramètre une liste de 5 cartes `main` et qui retourne `True` quand toutes les cartes sont de la même couleur.

In [12]:
def couleur(main):
    """
    >>> couleur([Carte(2,0), Carte(13,0), Carte(4,0), Carte(5,0), Carte(11,0)])
    True
    
    >>> couleur([Carte(2,0), Carte(13,2), Carte(4,0), Carte(5,0), Carte(11,0)])
    False
    """
    
    for i in range(len(main) - 1):
        if main[i].couleur != main[i + 1].couleur:
            return False
    return True
    
    
couleur([Carte(2,0), Carte(13,0), Carte(4,0), Carte(5,0), Carte(11,0)])
    

True

(Ici il faudrait parler de la moyenne empirique qui converge vers l'espérance)

* En simulant `nbr=100000` créations de jeux de cartes et pioche des 5 premières cartes, estimer la probabilité qu'une main aléatoire de 5 carte ne contienne que des cartes de la même couleur

In [13]:
res = 0
for _ in range(100000):
    JC = JeuDeCartes()
    M = JC.pioche_n(5)
    if couleur(M):
        res += 1
print(res / 100000)

0.00191


In [14]:
# 0.0019807923169267707 probabilité de la partie 1

* Comparer le résultat avec le résultat théorique trouvé à la partie 1.

<span style = "color : blue;">
    Les 2 probabilités sont assez similaires.
</span>

* Ecrire une fonction `tab_hauteurs(main)` qui retourne un tableau `T` de taille 15 tel que `T[i]` soit le nombre de cartes de hauteur `i` dans la main `main` (qui peut contenir n'importe quel nombre de cartes). On utilisera le fait que les valets sont de hauteur 11, les dames de hauteur 12, les rois de hauteur 13 et les as de hauteur 14.

Les cases `T[0]` et `T[1]` ne servent pas et valent toujours 0.

In [15]:
def tab_hauteurs(main):
    """
    >>> tab_hauteurs([Carte(2,0), Carte(2,1), Carte(4,0), Carte(11,0), Carte(11,3)])
    [0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]
    
    >>> tab_hauteurs([Carte(14,0), Carte(14,1), Carte(14,2)])
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
    """
    
    T = [0] * 15
    for j in range(len(main)):
        T[main[j].hauteur] += 1
    return T

tab_hauteurs([Carte(2,0), Carte(2,1), Carte(4,0), Carte(11,0), Carte(11,3)])


[0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]

* Ecrire une fonction `valeur_main(main)` qui renvoie un entier indiquant la meilleure combinaison présente dans une main `main` de cinq cartes, avec le numérotage des combinaisons indiqué ci-dessous. 

**Indication :** On pourra utiliser `tab_hauteurs(main)`.

In [52]:
#HAUTEUR = 0
#PAIRE = 1
#DOUBLE_PAIRE = 2
#BRELAN = 3
#QUINTE = 4
#COULEUR = 5
#FULL_HOUSE = 6
#CARRE = 7
#QUINTE_FLUSH = 8

def valeur_main(main):
    """
    >>> valeur_main([Carte(2,3), Carte(3,2), Carte(10,2), Carte(11,1), Carte(13,0)])
    0
    >>> valeur_main([Carte(2,3), Carte(3,2), Carte(11,2), Carte(11,1), Carte(13,0)])
    1
    >>> valeur_main([Carte(2,3), Carte(2,0), Carte(11,2), Carte(11,1), Carte(13,0)])
    2
    >>> valeur_main([Carte(2,3), Carte(11,0), Carte(11,2), Carte(11,1), Carte(13,0)])
    3
    >>> valeur_main([Carte(2,3), Carte(3,0), Carte(4,2), Carte(5,1), Carte(6,0)])
    4
    >>> valeur_main([Carte(2,3), Carte(3,0), Carte(4,2), Carte(5,1), Carte(14,0)])
    4
    >>> valeur_main([Carte(2,3), Carte(3,3), Carte(4,3), Carte(7,3), Carte(14,3)])
    5
    >>> valeur_main([Carte(2,3), Carte(11,0), Carte(11,2), Carte(11,1), Carte(2,0)])
    6
    >>> valeur_main([Carte(2,3), Carte(11,0), Carte(11,2), Carte(11,1), Carte(11,3)])
    7
    >>> valeur_main([Carte(10,0), Carte(11,0), Carte(12,0), Carte(13,0), Carte(14,0)])
    8
    """
     
    # mettre la liste en ordre avant de faire la comparaison pour verifier la Quinte
    lst_hauteur = []
    for carte in main :
        lst_hauteur.append(carte.hauteur) 
    lst_hauteur.sort() # range les hauteurs par ordre croissant
    
    suite = 0
    for i in range(len(lst_hauteur) - 1):           
        if (lst_hauteur[i] + 1 == lst_hauteur[i + 1]) or (lst_hauteur[i + 1] == 14 and lst_hauteur[0] == 2) :
            suite += 1
        else :
            suite = 0            
        if suite == 4 :
            if couleur(main) == True: # on a une quinte flush
                return 8
            return 4                  # on a une simple quinte

    if couleur(main) == False : # cas ou il n'y a pas de couleurs
        
        if max(tab_hauteurs(main)) == 1 : # hauteur
            return 0
        
        elif max(tab_hauteurs(main)) == 2 :
            occurence = tab_hauteurs(main).count(2) # compte le nombre de paire
            if occurence == 1 :   # une paire
                return 1
            elif occurence == 2 :  # deux paire
                return 2
            
        elif max(tab_hauteurs(main)) == 4 : # carre
            return 7
        
        elif max(tab_hauteurs(main)) == 3 :
            if max(x for x in tab_hauteurs(main) if x != 3) == 2 : # full house (brelan et paire)
                return 6
            return 3 # brelan

    elif couleur(main) == True : # cas ou il y a une couleur

        if max(tab_hauteurs(main)) == 1 : # couleur
            return 5

doctest.testmod()

**********************************************************************
File "__main__", line 5, in __main__.texas_holdem
Failed example:
    sorted(texas_holdem([Carte(2,0), Carte(7,0)], [Carte(3, 1), Carte(3,0), Carte(8,0), Carte(10, 2), Carte(11, 0)]))
Exception raised:
    Traceback (most recent call last):
      File "c:\users\selym\appdata\local\programs\python\python39-32\lib\doctest.py", line 1336, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest __main__.texas_holdem[0]>", line 1, in <module>
        sorted(texas_holdem([Carte(2,0), Carte(7,0)], [Carte(3, 1), Carte(3,0), Carte(8,0), Carte(10, 2), Carte(11, 0)]))
    TypeError: 'NoneType' object is not iterable
**********************************************************************
File "__main__", line 7, in __main__.texas_holdem
Failed example:
    sorted(texas_holdem([Carte(2,0), Carte(11,1)], [Carte(3, 1), Carte(3,0), Carte(8,0), Carte(10, 2), Carte(11, 0)]))
Exception raised:
    Traceba

TestResults(failed=2, attempted=18)

* En simulant 10000 jeux de cartes où on prend une main de 5 cartes à chaque fois, estimer la probabilité d'avoir une paire et celle d'avoir 2 paires.

In [69]:
proba_paire = 0
proba_deux_paires = 0
for _ in range(10000):
    JC = JeuDeCartes()
    M = JC.pioche_n(5)
    val = valeur_main(M)
    if val == 1 :
        proba_paire += 1
    if val == 2 :
        proba_deux_paires += 1
print(f"La probabilité d'avoir une paire est de {proba_paire / 100}%.")
print(f"La probabilité d'avoir deux paires est de {proba_deux_paires / 100}%.")

La probabilité d'avoir une paire est de 40.33%.
La probabilité d'avoir deux paires est de 4.08%.


## 4. Texas Holdem

Dans cette version du poker chaque joueur ne reçoit que deux cartes, et cinq cartes sont (progressivement) dévoilées sur la table. La valeur du jeu de chaque joueur est la meilleure combinaison possibles de cinq cartes qu'il peut faire en utilisant ses cartes et les cartes de la table (il peut prendre soit les 2 cartes de sa main et 3 de la table, soit 1 de sa main et 4 de la table, soit les 5 de la table).

On rappelle qu'on peut énumérer tous les sous-ensembles de taille $k$ d'un ensemble donné grâce à `combinations` de la bibliothèque `itertools` (voir document *Enumérer des collections d'éléments*)

Un des problèmes pour calculer la meilleure main c'est qu'il y a des règles précises pour, par exemple, départager deux mains qui ont toutes les deux une paire, ou deux full house, etc. Implanter ces règles et un peu laborieux, on pourra donc utiliser le code suivant qui fait le travail.

In [70]:
def compare_mains(main1, main2):
    """Retourne True si la meilleure main est main1 ou s'il y a égalité"""
    v1 = valeur_main(main1)
    v2 = valeur_main(main2)
    if v1 != v2:
        return v1 > v2
    # gestion des égalités
    if v1 == QUINTE or v1 == QUINTE_FLUSH:
        m1 = max([carte.hauteur for carte in main1])
        m2 = max([carte.hauteur for carte in main2])
        return m1 >= m2
    if v1 == HAUTEUR or v1 == COULEUR:
        tri1 = sorted([carte.hauteur for carte in main1], reverse=True)
        tri2 = sorted([carte.hauteur for carte in main2], reverse=True)
        return tri1 >= tri2
    th1 = tab_hauteurs(main1)
    th2 = tab_hauteurs(main2)
    if v1 == CARRE:
        h1 = th1.index(4)
        h2 = th2.index(4) 
        if h1 != h2: # hauteur du carré pour décider qui gagne
            return h1 >= h2
        return th1.index(1) >= th2.index(1) # si carré de meme hauteur 5eme carte départage
    if v1 == FULL_HOUSE:
        h1 = th1.index(3)
        h2 = th2.index(3)
        if h1 != h2: # hauteur du brelan pour décider qui gagne
            return h1 >= h2
        return th1.index(2) >= th2.index(2) # si brelan de meme hauteur, paire  départage
    if v1 == BRELAN:
        h1 = th1.index(3)
        h2 = th2.index(3)
        if h1 != h2: # hauteur du brelan pour décider qui gagne
            return h1 >= h2
        tri1 = sorted([carte.hauteur for carte in main1], reverse=True)
        tri2 = sorted([carte.hauteur for carte in main2], reverse=True)
        return tri1 >= tri2
    if v1 == PAIRE:
        h1 = th1.index(2)
        h2 = th2.index(2)
        if h1 != h2: # hauteur de la paire pour décider qui gagne
            return h1 >= h2
        tri1 = sorted([carte.hauteur for carte in main1], reverse=True)
        tri2 = sorted([carte.hauteur for carte in main2], reverse=True)
        return tri1 >= tri2
    if v1 == DOUBLE_PAIRE:
        paires1 = sorted([i for i in range(15) if th1[i]==2], reverse=True)
        paires2 = sorted([i for i in range(15) if th1[i]==2], reverse=True)
        if paires1 != paires2:
            return paires1 >= paires2
        return th1.index(1) >= th2.index(1)

* Ecrire une fonction `texas_holdem(main, table)` qui prend en argument une main de 2 cartes `main` et une `table` de cinq cartes, et qui retourne la meilleure combinaison de 5 cartes que l'on peut faire avec cette main.

In [84]:
from itertools import combinations

def texas_holdem(main, table):
    """
    >>> sorted(texas_holdem([Carte(2,0), Carte(7,0)], [Carte(3, 1), Carte(3,0), Carte(8,0), Carte(10, 2), Carte(11, 0)]))
    [2-Trefle, 3-Trefle, 7-Trefle, 8-Trefle, V-Trefle]
    >>> sorted(texas_holdem([Carte(2,0), Carte(11,1)], [Carte(3, 1), Carte(3,0), Carte(8,0), Carte(10, 2), Carte(11, 0)]))
    [3-Carreau, 3-Trefle, 10-Coeur, V-Carreau, V-Trefle]   
    """
    pass
texas_holdem([Carte(2,0), Carte(7,0)], [Carte(3, 1), Carte(3,0), Carte(8,0), Carte(10, 2), Carte(11, 0)])

On souhaite maintenant savoir si une main de deux cartes que l'on reçoit au début d'une partie, avant que la table ne soit dévoilée, est une bonne main ou non.

* Ajouter une méthode `enlever_carte(self, carte)` à la classe `JeuDeCarte`, qui permet d'enlever la carte `carte` du jeu. 

In [20]:
class JeuDeCartes:
    def __init__(self):
        # à compléter
                
    def pioche(self):
        # à compléter

    def pioche_n(self, n):
        # à compléter
    
    def enlever_carte(self, carte):
        # à compléter

IndentationError: expected an indented block (328941340.py, line 5)

* Ecrire une fonction `simulation(main, nbr)` qui simule `nbr` parties à deux joueurs de Poker Texas Holdem où le premier joueur possède la main de départ `main` (de deux cartes) et qui retourne le nombre de parties gagnées par le premier joueur (ou avec égalité).

* En utilisant `simulation` avec `nbr=10000`, estimer les probabilités de gain des mains suivantes :
- une paire d'as
- un as et un deux de couleurs différentes
- un as et un deux de même couleur
- une paire de deux
- un deux et un sept de couleurs différentes
- un deux et un sept de même couleur

(ça prend un peu de temps, soyez patients)

## 5. Génération des combinaisons

Pour générer toutes les combinaisons (ou sous-ensembles) d'un ensemble, nous avons utilisé la fonction `combinations` du module `itertools`. Nous allons maintenant programmer nous-mêmes cette génération.

- Ecrire une fonction `subset` qui génère tous les sous-ensembles de cardinal donné d'un ensemble fini (sous forme de listes).

~~~py
>>> subset([1, 2, 3], 2)

[[1, 2], [1, 3], [2, 3]]
~~~

On pourra procéder de manière récursive en utilisant la propriété suivante.

Soit $A$ un ensemble non-vide. Soit $x$ un élément de $A$. Alors tout sous-ensemble de cardinal $n$ de $A$ est :

* soit un ensemble de la forme $\{x\} \cup S$, où $S$ est un sous-ensemble de cardinal $n-1$ de $A \setminus \{x\}$ ;
* soit un sous-ensemble de cardinal $n$ de $A \setminus \{x\}$.